use simplified data 2

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [19]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME, encoding='ANSI')
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound, label_left_bound, data_right_bound=None, label_right_bound=None):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    if label_right_bound is None:
        label_right_bound = len(data.columns)
    if data_right_bound is None:
        data_right_bound = label_left_bound
    X = data.iloc[1:, list(range(data_left_bound, data_right_bound))]
    y = data.iloc[1:, list(range(label_left_bound, label_right_bound))]
    
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

def SplitNparr(original_arr: np.ndarray, train_portion: float)->tuple:
    data_len =len(original_arr)
    train_len = int(data_len * train_portion)
    indices = list(range(data_len))
    random.shuffle(indices)
    train_idx = indices[:train_len]
    validate_idx = indices[train_len:]
    training_arr  = original_arr[train_idx]
    validation_arr = original_arr[validate_idx]

    return (training_arr,validation_arr)


def Split1Df(data: pd.DataFrame,  train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - data: any data
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - data_train: data for the training set.
    - data_val: data for the validation set.

    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(data)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]


    data_train, data_val = data.loc[train_idx], data.loc[val_idx]
   
    return data_train, data_val

def SplitBothXy_Df(X: pd.DataFrame, y: pd.DataFrame, train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - X: Features 
    - y: Target variable
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - X_train: Features for the training set.
    - X_val: Features for the validation set.
    - y_train: Target variable for the training set.
    - y_val: Target variable for the validation set.
    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(X)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]
    print("type of val_idx", type(val_idx))
    print("type of train_idx", type(train_idx))
    print("len of val_idx", len(val_idx))

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert(X_train.shape[1] == X_val.shape[1])
    assert(y_train.shape[1] == y_val.shape[1])
    return X_train, X_val, y_train, y_val



In [3]:
def DeleteMedicine(y):
    threshold = 250
    for col in y.columns:
        # print(f'Number of {col} is {y[col].sum()}.')
        if y[col].sum() < threshold:
            y = y.drop(col, axis=1)
    # Debug
    print(f'DeleteMedicine: shape of y is {y.shape}.')
    return y

In [4]:
data_pd = ReadData("./simplified_data/simplified_data2.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).


In [20]:
first_medicine_idx = None
#first_medicine_idx = 113
first_medicane = '麻黃'

for i in range(len(data_pd.columns.tolist())):
    if data_pd.columns.tolist()[i] == first_medicane:
        first_medicine_idx = i
        break
print("first_medicine: ",data_pd.columns[first_medicine_idx])
print("first_medicine_idx: ", first_medicine_idx)
print("should be 麻黃")

# split data into X and y
# x= all symptoms, diagnosis, body status
# y= all medince
X,y = SplitXY(data_pd, data_left_bound=2, data_right_bound=first_medicine_idx, label_left_bound=first_medicine_idx)

# X is a df, split X into trainX and testX
train_X, val_X, train_y, val_y = SplitBothXy_Df(X, y, 0.8)


print("train_X.shape: ", train_X.shape)
print("train_y.shape: ", train_y.shape)

first_medicine:  麻黃
first_medicine_idx:  113
should be 麻黃
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1   1   0   0
5    0   1   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    

In [18]:

X_np = train_X.values.astype('float64')
X_val_np = val_X.values.astype('float64')
print("X transformed to np array")
print("type of X_np:", type(X_np))
print("shape of X_np:", X_np.shape)
print("type of X_val_np:", type(X_val_np))
print("shape of X_val_np:", X_val_np.shape)
num_col_x = X_np.shape[1]
print("number of col in (train) x:",num_col_x)
print("shape of train y:", train_y.shape)



X transformed to np array
type of X_np: <class 'numpy.ndarray'>
shape of X_np: (637, 111)
type of X_val_np: <class 'numpy.ndarray'>
shape of X_val_np: (159, 111)
number of col in (train) x: 111
shape of train y: (637, 102)


In [16]:
# build and compile model

model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1792      
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,402
Trainable params: 2,402
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [29]:
all_res=[]
result_df_dict={}   # dict of df
loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine

for i in range(train_y.shape[1]):
#for i in range(2):
    res = {}
    res_list =[]
    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print("processing medicine:", chosen_col.name)
    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    # fit model for this medicine
    model.fit(x=X_np, y=chosen_y_np,
          batch_size=10, epochs=10, shuffle=True, verbose=0)
    
    # make prediction for this medicine
    predictions = model.predict(X_val_np)
    print(type(predictions))
    print("predictions.shape:", predictions.shape)
    
    df_predictions = pd.DataFrame(predictions, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    df_predictions["ground truth"] = y_val_chosen_col.values
     # create new column that checks if "predicted value" is equal to "ground truth"
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]

    accuracy = df_predictions["is_correct"].mean()
    loss_acc_dict[chosen_col.name] = accuracy  
    result_df_dict[chosen_col.name] = df_predictions
    

print("training done.")




processing medicine: 麻黃
5/5 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 桂枝
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 荊芥
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 防風
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 細辛
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 白芷
5/5 [==============================] - 0s 508us/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 生薑
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 辛夷
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
prediction

In [30]:

folder_name = "train_1_med_result"
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    print(df)
    df.to_csv(f"./{folder_name}/{key}_predictions.csv")




DataFrame for 麻黃:
     predicted as 0  predicted as 1  predicted value  ground truth  is_correct
0          0.823550        0.176450                0             0        True
1          0.993043        0.006957                0             0        True
2          0.824465        0.175535                0             0        True
3          0.824465        0.175535                0             0        True
4          0.824465        0.175535                0             0        True
..              ...             ...              ...           ...         ...
154        0.496263        0.503737                1             0       False
155        0.825839        0.174161                0             0        True
156        0.824465        0.175535                0             0        True
157        0.824465        0.175535                0             0        True
158        0.825839        0.174161                0             1       False

[159 rows x 5 columns]
DataFrame 

In [31]:
print(loss_acc_dict)

{'麻黃': 0.7484276729559748, '桂枝': 0.46540880503144655, '荊芥': 0.9559748427672956, '防風': 0.9119496855345912, '細辛': 0.7295597484276729, '白芷': 0.9685534591194969, '生薑': 0.660377358490566, '辛夷': 0.9433962264150944, '葛根': 0.9811320754716981, '升麻': 0.9748427672955975, '柴胡': 0.660377358490566, '蟬蛻': 0.9622641509433962, '石膏': 0.8427672955974843, '知母': 0.8176100628930818, '梔子': 0.89937106918239, '天花粉': 0.9811320754716981, '夏枯草': 0.9874213836477987, '決明子': 0.9748427672955975, '生地': 0.9119496855345912, '牡丹皮': 0.9874213836477987, '連翹': 0.9559748427672956, '射干': 0.9622641509433962, '黃連': 0.8113207547169812, '黃芩': 0.610062893081761, '黃柏': 0.9559748427672956, '龍膽草': 0.8427672955974843, '大黃': 0.8364779874213837, '芒硝': 0.9119496855345912, '防己': 0.9182389937106918, '秦艽': 1.0, '絡石藤': 0.9811320754716981, '蒼朮': 0.9119496855345912, '草果': 1.0, '茯苓': 0.6415094339622641, '豬苓': 0.8616352201257862, '薏苡仁': 0.9874213836477987, '海金砂': 1.0, '澤瀉': 0.6352201257861635, '車前子': 0.9937106918238994, '滑石': 0.9622641509433962,

In [36]:
import my_utilities as myutil
import json
myutil.dict_to_txt(loss_acc_dict, save_path=f"./{folder_name}/")
with open('./result/res.txt', 'w', encoding='utf-8') as file:
    json.dump(loss_acc_dict, file, indent=2)